# SPKIR CG data deep dive 1.1
### Ref Des: CP01CNSM-RID26-08-SPKIRB000

In [10]:
import numpy as np
import pandas as pd
import xarray as xr
import os
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from ooi_data_explorations.common import get_annotations, add_annotation_qc_flags
import ooi_data_explorations.common as common

from ooinet import M2M

from qartod_testing.timeseries import nanfill_time_gaps

In [3]:
# Set parameters for data request 
refdes = 'CP01CNSM-RID26-08-SPKIRB000'
site, node, sensor = refdes.split('-', 2)

In [21]:
methods = common.list_methods(site, node, sensor)
streams = []
for method in methods:
    streams.append(common.list_streams(site, node, sensor, method))
    
    # for stream in streams:
    #     r = common.m2m_request(site, node, sensor, method, stream)
    #     data = common.m2m_collect(r)


In [20]:
# View deployment information
deployments = M2M.get_deployments(refdes)
deployments

,deploymentNumber,uid,assetId,latitude,longitude,depth,deployStart,deployEnd,deployCruise,recoverCruise
0,1,CGINS-SPKIRB-00238,2387,40.13678,-70.76978,7.0,2013-11-21 18:16:00,2014-04-18 10:33:00,KN214,KN217
1,2,CGINS-SPKIRB-00272,1387,40.1339,-70.7789,7.0,2014-12-13 18:47:00,2014-12-15 20:58:00,KN224,KN224
2,3,CGINS-SPKIRB-00272,1387,40.14022,-70.77128,7.0,2015-05-07 17:34:00,2015-10-23 19:40:00,AT27,AT31
3,4,CGINS-SPKIRB-00291,1327,40.13323,-70.77843,7.0,2015-10-23 18:49:00,2016-04-04 12:03:00,AT31,AR1-07
4,5,CGINS-SPKIRB-00270,1390,40.14037,-70.77133,7.0,2016-05-13 13:50:00,2016-10-13 19:34:00,AR4,AR8
5,6,CGINS-SPKIRB-00291,1327,40.13342,-70.77847,7.0,2016-10-13 18:36:00,2017-06-09 16:05:00,AR8,AR18
6,7,CGINS-SPKIRB-00270,1390,40.139817,-70.77115,7.0,2017-06-09 14:24:00,2017-11-01 20:33:00,AR18,AR24
7,8,CGINS-SPKIRB-00275,1401,40.133383,-70.7783,7.0,2017-10-29 14:15:00,2018-03-29 19:37:00,AR24,AR28
8,9,CGINS-SPKIRB-00299,1337,40.13975,-70.77128,7.0,2018-03-24 21:32:00,2018-10-29 12:31:00,AR28,AR31
9,10,CGINS-SPKIRB-00271,1392,40.133367,-70.7777,7.0,2018-10-30 01:48:00,2019-04-07 18:08:00,AR31,AR34


In [9]:
# Define path on local machine to downloaded data
out_path = os.path.join(os.path.expanduser('~'), 'ooidata/m2m/', site.lower())
out_path = os.path.abspath(out_path)
if not os.path.exists(out_path):
    os.makedirs(out_path)
data_file = refdes + '.nc'
file_path = os.path.join(out_path, data_file)

if os.path.exists(file_path):
    # Open the local dataset if it exists
    data = xr.load_dataset(file_path)
else:
    # Download and save data to a netcdf file to ooidata directory in user root
    # data = combine_delivery_methods(site, node, sensor)                       # I need a method other than combine delivery methods because the function i tried to import was strictly for 
#                                                                               #  CTDBP instruments. There probably isn't a combine_delivery_methods for SPKIR yet.
    data.to_netcdf(file_path, mode='w', format='NETCDF4', engine='h5netcdf')

AttributeError: 'NoneType' object has no attribute 'reset_coords'

In [ ]:
# View annotations
annotations = get_annotations(site, node, sensor)
annotations = pd.DataFrame(annotations)
annotations

In [ ]:
if not annotations.empty:
    annotations = annotations.drop(columns=['@class'])
    annotations['beginDate'] = pd.to_datetime(annotations.beginDT, unit='ms').dt.strftime('%Y-%m-%dT%H:%M:%S')
    annotations['endDate'] = pd.to_datetime(annotations.endDT, unit='ms').dt.strftime('%Y-%m-%dT%H:%M:%S')

    # create an annotation-based quality flag
    data_edit = add_annotation_qc_flags(data, annotations)

    # clean-up the data, removing values that were marked as fail either from the quality checks or in the
    # annotations, and all data collected after the cut off date
    data_edit = data_edit.where(data_edit.rollup_annotations_qc_results != 4)

In [ ]:
dataset_full = nanfill_time_gaps(data_edit)
dataset_full